# iris CART using DecisionTree.jl
Loading packages that we will use

In [1]:
using DecisionTree
using Random, Statistics
using DataFrames

Loading Data (Iris dataset)

In [17]:
X, y = load_data("iris")
DF1 = DataFrame(X, :auto)
DF2 = DataFrame("Type" => y )
DF = DataFrame(x1 = float.(DF1[!,1]),
x2 = float.(DF1[!,2]),
x3 = float.(DF1[!,3]),
x4 = float.(DF1[!,4]),
y = string.(DF2[!,1]));
Matrix(DF[!,1:4]);


Now using pycall to split data into training and testing sets

In [18]:
using PyCall
ModelSelection = pyimport("sklearn.model_selection")
X_train, X_test, y_train, y_test =
 ModelSelection.train_test_split(Matrix(DF[!,1:4]),DF[!,5]);

Decision Tree

In [26]:
model = DecisionTreeClassifier(max_depth = 2);
fit!(model, X_train, y_train);
#Printting the tree
print_tree(model)

Feature 4 < 0.8 ?
├─ Iris-setosa : 40/40
└─ Feature 3 < 4.75 ?
    ├─ Iris-versicolor : 35/36
    └─ Iris-virginica : 32/36


In [43]:
train = [X_train y_train]
# view decision node data subset

train_R = train[train[:, 4] .> 0.8, :];


Ready to make some predictions

In [47]:
y_hat = predict(model, X_test)
#and checking the accuracy
accuracy = mean(y_hat .== y_test)

0.9473684210526315